In [2]:
import pyodbc
import re
import getpass
from datetime import datetime
 
server = '172.10.2.5'
database = 'Car Store DB'
username = 'Carstore'
password = 'nss@12345'
connection = pyodbc.connect("DRIVER={SQL Server Native Client 11.0};SERVER=" + server + ';DATABASE=' + database + ';UID=' + username + ';PWD=' + password)
 
text = "-Welcome to Car Shop!-"
formatted_text = f"\033[1m{text.center(80)}\033[0m"
print(formatted_text)
 
# Function to save user data to the database(users table)
 
def save_user(connection, username, password, fullname, email, gender, phonenumber, date_of_birth, address, city, state ):
    cursor = connection.cursor()
    registration_timestamp = datetime.now()
    cursor.execute(
        "INSERT INTO users (username, password,fullname,email, gender, phonenumber, date_of_birth, address, city, state, registration_timestamp) "
        "VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?,?,?)",
        (username, password, fullname, email, gender, phonenumber, date_of_birth, address, city, state ,registration_timestamp))
    connection.commit()
 
# check if the user already exists in the database
 
def user_exists(connection, username):
    cursor = connection.cursor()
    cursor.execute("SELECT username FROM users WHERE username=?", (username,))
    result = cursor.fetchone()
    return result is not None
 
 
# Condition for password strength
 
def is_strong_password(password):
    return (
            len(password) >= 8 and
            re.search(r"[A-Z]", password) and
            re.search(r"[0-9]", password) and
            re.search(r"[!@#$%^&*]", password)
    )
 
# user login
 
def login(connection):
    print("Login:")
    username = input("Enter username: ")
    password = getpass.getpass("Enter password: ")
    cursor = connection.cursor()
    cursor.execute("SELECT u_id FROM users WHERE username = ? AND password = ?", (username, password))
    result = cursor.fetchone()
    if result is not None:
        u_id = result[0]
        last_login_timestamp = datetime.now()
        cursor.execute("UPDATE users SET last_login_timestamp = ? WHERE u_id = ?", (last_login_timestamp, result[0]))
        connection.commit()
        print("Login successful. Welcome, " + username + "!")
        cursor.execute("SELECT payment_id, car_name, bank_name, car_price, payment_method FROM car_selection WHERE u_id = ?", (u_id,))
        purchase_history = cursor.fetchall()
        if purchase_history:
            print("Previous Purchase History:")
            for purchase in purchase_history:
                payment_id, car_name, bank_name, car_price, payment_method = purchase
                if payment_method == 'Full':
                    print(f"You have purchased a {car_name} for ${car_price:.2f} and paid through {bank_name}.")
                elif payment_method == 'EMI':
                    cursor.execute("""
                SELECT e.emi_amount_per_month
                FROM car_selection cs
                JOIN Emi e ON cs.payment_id = e.payment_id
                WHERE cs.payment_method = 'EMI' AND cs.u_id = ? AND cs.car_name = ? AND cs.bank_name = ?""", (u_id, car_name, bank_name))
 
                    emi_data = cursor.fetchone()
                    if emi_data:
                        emi_amount_per_month = emi_data[0]
                        print(f"You have purchased a {car_name} for ${car_price:.2f} and are paying an EMI of ${emi_amount_per_month:.2f} through {bank_name}.")
        else:
            print("No Purchase History Found")
 
        %run car_selection.ipynb
    else:
        text2 = "Login failed. Invalid username or password."
        formatted_text = f"\033[91m{text2}\033[0m"
        print(formatted_text)

 
#for gender
def get_valid_gender_input():
    while True:
        gender = input("Enter gender (F, M, or O): ").strip().lower()
        if gender in ['f', 'm', 'o']:
            return gender
        else:
            print("Invalid gender. Please enter 'F' for female, 'M' for male, or 'O' for other.")
#for phone number           
def check_phonenumber():
    while True:
        try:
            phonenumber = input("Enter phone number: ")
            if len(phonenumber) == 10 and phonenumber.isdigit():
                return phonenumber
            else:
                print("Enter a 10-digit numeric phone number.")
        except ValueError:
            print("Phone number should be numeric.")
#for date of birth
def check_dob():
    while True:
        try:
            date_of_birth = input("Enter date of birth (YYYY-MM-DD): ")
            date_of_birth = datetime.strptime(date_of_birth, '%Y-%m-%d').date()
            return date_of_birth
            break
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD.")
            continue
def get_valid_email_input():
    while True:
        email = input("Enter email: ")
        pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
        if re.match(pattern, email):
            return email
        else:
            print("Invalid email. Please enter a valid email address.")
            continue
# registering a new user
def register(connection):
    try:
        print("Register a new user:")
        username = input("Enter username: ")
        if user_exists(connection, username):
            raise ValueError("User already exists. Please choose a different name.")
        password = getpass.getpass("Password: ")
        while True:
            if is_strong_password(password):
                fullname = input("Enter Full Name:")
                email = get_valid_email_input()
                gender = get_valid_gender_input()
                phonenumber = check_phonenumber()
                date_of_birth = check_dob()
                address = input("Enter address: ")
                city = input("Enter city: ")
                state = input("Enter State: ")
                save_user(connection, username, password, fullname, email, gender, phonenumber, date_of_birth, address, city, state)
                print("User registered successfully.")
                break
            else:
                raise ValueError("Password is not strong enough. It should be at least 8 characters long, contain at least one uppercase letter, one digit, and one special character (!@#$%^&*). Please try again.")
    except ValueError as ve:
        formatted_text = f"\033[91m{ve}\033[0m"
        print(formatted_text)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
# exit
def exit(connection):
    connection.close()
 
 
def main():
    while True: 
        print("1. Register\n2. Login\n3. Exit")
        choice = input("Enter your choice: ")
        if choice == "1":
            register(connection)
        elif choice == "2":
            login(connection)
        elif choice == "3":
            exit(connection)
            break
        else:
            print("Invalid choice.")
 
if __name__ == "__main__":
    main()

                             -Welcome to Car Shop!-                             
1. Register
2. Login
3. Exit
Enter your choice: 2
Login:
Enter username: himansh
Enter password: ········
Login successful. Welcome, himansh!
No Purchase History Found
                                 -Car Options-                                  

    carid    car_company  car_name  car_model  car_mileage car_color  \
0      1         Toyota     Camry      sedan         30.5      Blue   
1      2          Honda     Civic      sedan         35.2       Red   
2      3           Ford  Explorer        Suv         23.8     Black   
3      4      Chevrolet     Cruze      sedan         28.1    Silver   
4      5           Ford  Ecosport        Suv         25.1     Black   
5      6  Maruti Suzuki     Swift  Hatchback         26.1     White   

   car_price  stock  
0   150000.0     10  
1   200000.0      9  
2   350000.0      8  
3   180000.0     10  
4   180000.0      8  
5   700000.0      8  
Enter the carid 